In [ ]:
# /// script
# requires-python = ">=3.13"
# dependencies = [
#     "corduroy @ git+https://github.com/norlandrhagen/corduroy",
#     "bokeh",
#     "dask",
#     "distributed",
#     "ipykernel",
#     "jupyter",
#     "matplotlib",
#     "rioxarray",
#     "zarr",
# ]
# ///

from distributed import Client
import xarray as xr
import xproj  # noqa ignore
import corduroy  # noqa ignore
import matplotlib.pyplot as plt

In [ ]:
client = Client(n_workers=4)
client

In [ ]:
bucket_url = "s3://prd-tnm/"
prefix = "StagedProducts/Elevation/13/TIFF/USGS_Seamless_DEM_13.vrt"
file_url = f"{bucket_url}{prefix}"

# subset to part of Glacier National Park
bbox = [-113.973541, 48.468389, -113.383713, 48.819721]
ds = xr.open_dataset(file_url, engine="rasterio", chunks="auto").sel(
    x=slice(bbox[0], bbox[2]), y=slice(bbox[3], bbox[1])
)
ds = ds.isel(band=0).rename({"band_data": "DEM"}).drop_vars("band")


ds = ds.proj.assign_crs(spatial_ref="EPSG:4326", allow_override=True)
ds

In [ ]:
hillshade = ds["DEM"].dem.hillshade()
slope = ds["DEM"].dem.slope()
aspect = ds["DEM"].dem.aspect()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(24, 20), sharex=True, sharey=True)

ds["DEM"].plot(ax=axes[0, 0], cmap="terrain", add_colorbar=True)
axes[0, 0].set_title("Input DEM")

slope.plot(ax=axes[0, 1], cmap="magma")
axes[0, 1].set_title("Slope")

aspect.plot(ax=axes[1, 0], cmap="twilight")
axes[1, 0].set_title("Aspect")

hillshade.plot(ax=axes[1, 1], cmap="gray")
axes[1, 1].set_title("Hillshade")

plt.tight_layout()